# BrainQuant3D Setup
## 1. Download necessary applications/packages
Ensure you have these downloaded on your computer:
1. git (for setting up BrainQuant3D as a package): https://git-scm.com/downloads
2. anaconda/miniconda (for isolating packages in its own Python environment): https://www.anaconda.com/download

Then, open your terminal (mac, linux) or anaconda prompt (windows) and execute these commands:
```
cd <path to directory where this notebook and the .yml file is in> #e.g. cd mnt/ssd
conda env create -f bq3d_env.yml
source activate bq3d_env
pip install git+https://github.com/sunilgandhilab/brainquant3d.git
```
then, choose the newly created "bq3d" conda environment as the kernel to run these Jupyter Notebooks and code.

In [26]:
#run this code block to ensure everything has been imported correctly.
import bq3d

import os
from pathlib import Path
import yaml
import bq3d
import glob

## 2. Download Allen Brain Atlas datasets
Download Warping.zip, which contains the Allen Brain Atlas datasets (4.04 GB zipped, 39.79 GB unzipped):

Option 1 - Download from Google Drive:
https://drive.google.com/drive/u/2/folders/1JjUxW3k2foIB2LBOa5LrtnCTofHToVho

Option 2 - Download using the following command:
```
$ wget --no-check-certificate https://glams.bio.uci.edu/Warping.zip
```
Unzip and move the downloaded Warping directory to a location you will remember (e.g. SSD drive).
```
$ unzip Warping.zip
$ mv Warping /mnt/ssd
```
Update this path in the parameter_template.py file in the same folder.

In [27]:
brain_atlas_directory = "/Users/ytcao/Documents/Warping" #!!! modify the path to the downloaded brain atlas directory on your computer

try:
    if not any(Path(brain_atlas_directory).iterdir()):
        raise Exception("No files found in the brain atlas directory. Make sure that the path is correct!")
except Exception as e: print(e)

print(f"\nreplace this variable in parameter_template.py (line 37):\nPathReg = Path('{brain_atlas_directory}')")


replace this variable in parameter_template.py (line 37):
PathReg = Path('/Users/ytcao/Documents/Warping')


## 3. Set up configuration file
Now you need to set up the configuration file. This file will tell BrainQuant3D where to place temp data and where to find the atlas data. The configuration template file, default.conf, can be found in the package install location.

Temp_path: This is the location where temp data will be stored. It should be on a high-performance SSD. Preferably an NVMe. If the path does not exist, it will be created.

Rigid_default, Affine_default, BSpline_default, Labeled_default, Annotations_default: These are the paths to the Allen Brain Atlas datasets downloaded earlier.

Processing_cores, Thread_ram_max_Gb: These parameters specify how many processing cores to use and how much RAM to user per core. For example, if your system has 10 cores and 256 GB of RAM, you would set "Processing_cores" to 10 and "Thread_ram_max_Gb" to a value such as 22. This would use 10 x 22 = 220 GB of total RAM. It's a good idea to leave a little available RAM so as to not overload the system.

Adjust variables as necessary below:

In [24]:
### locate temporary directory path on your computer
!echo "temporary directory:" $TMPDIR

### uncomment to find the number of cores and amount of RAM supported by your computer (linux)
# !lscpu | egrep 'Model name|Socket|Thread|NUMA|CPU\(s\)' #look for number of CPUs
# !free -h #max RAM

### for windows:
# !wmic cpu get caption, deviceid, name, numberofcores, maxclockspeed, status #look for number of cores
# !systeminfo #look for max RAM

### for mac:
!sysctl -a | grep hw.ncpu #number of cores
!sysctl -n hw.memsize #max RAM in bytes; divide by (1024**3) for GB
print(8589934592/(1024**3), "GB RAM")

temporary directory: /var/folders/47/6bm96nxd0ks5x370bsvkpghh0000gn/T/
hw.ncpu: 8
8589934592
8.0 GB ram


In [ ]:
### replace temp_directory variable with above output (optional), or at any location you choose.

temp_directory = '' #!!! change this accordingly-- can be temporary directory path of your computer, the scratch folder, any directory you'd like. for example, on mac: temp_directory = '/var/folders/47/6bm96nxd0ks5x370bsvkpghh0000gn/T/'

processing_cores = 1 #!!! change this accordingly
thread_ram_max_GB = 4 #!!! change this accordingly

#find installation address of bq3d package
bq3d_package_directory = Path(bq3d.__file__).parent
assert Path(bq3d_package_directory, "default.conf").exists(), f"cannot find BrainQuant3D configuration file in the directory {bq3d_package_directory}"
print("BQ3D_PACKAGE_DIRECTORY:", bq3d_package_directory)

run the following code blocks to automatically create a configuration file:

In [ ]:
### create updated config file
try:
    parigid_path = Path(brain_atlas_directory, "ParRigid.txt")
    paraffine_path = Path(brain_atlas_directory, "ParAffine.txt")
    parbspline_path = Path(brain_atlas_directory, "ParBSpline.txt")
    annotation_path = Path(brain_atlas_directory, "ARA2", "annotation_25_right.tif")
    collapse_path = Path(brain_atlas_directory, "ARA2_annotation_info_collapse.csv")
except Exception as e: print(e, f"file does not exist in the provided directory {brain_atlas_directory}")

config_data = {
    'user': {
        'default': {
            #paths
            ## path for mac only
            ## 'Ilastik_path': f'{str(bq3d_package_directory)}/.external/ilastik-1.3.3/Contents/ilastik-release', 
            'Ilastik_path': f'{str(bq3d_package_directory)}/.external/ilastik-1.3.3',
            'Elastix_path': f'{str(bq3d_package_directory)}/.external/elastix-5.0.0',
            'Temp_path':    temp_directory, 

            #brain atlas
            'Rigid_default':       f'{str(parigid_path)}',
            'Affine_default':      f'{str(paraffine_path)}',
            'BSpline_default':     f'{str(parbspline_path)}',
            'Labeled_default':     f'{str(annotation_path)}',
            'Annotations_default': f'{str(collapse_path)}',
            'Console_level':       'verbose',

            #processing
            'Processing_cores':    processing_cores, 
            'Thread_ram_max_Gb':   thread_ram_max_GB, 
        }
    }
}
print(config_data)

In [ ]:
### save to the bq3d folder
with open(Path(bq3d_package_directory,"brainquant3d.conf"), mode="w+") as f:
    yaml.dump(config_data, f, sort_keys=False, indent=4, allow_unicode=True)

You can now move onto the tutorial Jupyter Notebook (01_bq3d_tutorial.ipynb).